In [ ]:
import numpy as np
import pandas as pd
import folium
import pymongo
from pymongo import MongoClient
client=MongoClient('localhost',27017)
db=client.Companies
import folium
from folium import plugins
from folium.plugins import HeatMap
import math
import gmplot

In [ ]:
#Lista con las únicas categorias que hay en toda la base
kind=[e for e in db.Companies.find().distinct('category_code')]

In [ ]:
def validation_companies():
    i=0
    while i==0:
        j=0
        x=input("Enlista, por comas y de acuerdo a la lista anterior, las categorías de las compañias que la Startup va a necesitar: ")
        ent=x.split(",")
        for e in ent:
            if e in kind:
                print("{} si está en la lista".format(e))
                j+=1
            else:
                print("{} no está en la lista".format(e))       
                i=0
        if j==len(ent):i=1
    return ent           

def validation_principal(ent):
    i=0
    while i==0:
        x=input("Ingresa la categoría de la compañia que es principal para la startup:")
        if x in kind and x in ent:
            print("{} si está en la lista".format(x))
            i=1
        else:
            print("{} no está en la lista".format(x))       
            i=0    
    return x        
                
#entradas=validation_companies()
#principal=validation_principal(entradas)

In [ ]:
#Funcion que calcula la distancia de dos puntos
def dist(p1,p2):
    x1,x2,y1,y2=p1[0],p2[0],p1[1],p2[1]
    dist=math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return round(dist,2)

In [ ]:
#Listas de todas las compañias de acuerdo al tipo que le des, typ es la categoria
def lstcompañias(typ):
    return list(db.Companies.find({'category_code':typ},{'_id':0,'name':1,'offices.latitude':1,'offices.longitude':1}).sort('name',pymongo.ASCENDING))

In [ ]:
#compañias[0] es una lista con todas las oficinas de las compañias de la primer categoria ingresada
compañias=list(map(lstcompañias,entradas))

In [ ]:
#Lista con las latitudes de las compañias que ingresas, compañia[i] es cada categoria ingresada
def lats(compañia):
    dirtlat=[e['offices'][f]['latitude'] for e in compañia for f in range(len(e['offices']))]
    lat=[e for e in dirtlat if e!=None]
    return lat

In [ ]:
#latitudes[0] son las latitudes de todas las oficinas de la compañia referida en compañia[0]
latitudes=list(map(lats,compañias))

In [ ]:
#Lista con las longitudes de las compañias que ingresas, compañia[i] es cada categoria ingresada
def lons(compañia):
    dirtlon=[e['offices'][f]['longitude'] for e in compañia for f in range(len(e['offices']))]
    lon=[e for e in dirtlon if e!=None]
    return lon

In [ ]:
#longitudes[0] son las longitudes de todas las oficinas de la compañia referida en compañia[0]
longitudes=list(map(lons,compañias))

In [ ]:
#Lista de coordenadas de las oficinas de la categoria seleccionada
def coord(la,lo):
    loc=list(zip(la,lo))
    return loc

In [ ]:
#coordenadas[0] son las coordenadas de las oficinas de  de las compañia referida en compañia[0]
coordenadas=[coord(latitudes[e],longitudes[e]) for e in range(len(entradas))]

In [ ]:
base=entradas.index(principal)

In [ ]:
coordenadas_principal=coordenadas[base]

In [ ]:
del coordenadas[base]
del entradas[base]

In [ ]:
#Funcion que calcula la distancia de la oficina principal a otra especificada
def dist_principal(p2):
    p1=coordenadas_principal[0]
    x1,x2,y1,y2=p1[0],p2[0],p1[1],p2[1]
    dist=math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return round(dist,2)

In [ ]:
#Lista de distancias minimas de la oficina principal a una oficina de cada categoría
distancias_minimas=[min(list(map(dist_principal,coordenadas[e]))) for e  in range(len(coordenadas))]

In [ ]:
#Lista de indices de las coordenadas que corresponden al punto mas cercano a la oficina principal
indices=[list(map(dist_principal,coordenadas[e])).index(distancias_minimas[e]) for e in range(len(distancias_minimas))]

In [ ]:
#Etiqueta de la oficina principal y etiquetas de las demas oficinas indicadas en las categorias
labelprincipal=list(db.Companies.find({'$and':[{'category_code':principal},{'offices.latitude':coordenadas_principal[0][0]}]},{'_id':0,'name':1,'category_code':1}))
labels=[list(db.Companies.find({'$and':[{'category_code':entradas[e]},{'offices.latitude':coordenadas[e][indices[e]][0]}]},{'_id':0,'name':1,'category_code':1})) for e in range(len(entradas))]

In [ ]:
#Funcion que determina el punto medio, dado dos puntos
def punto_medio(p1,p2):
    x1,x2,y1,y2=p1[0],p2[0],p1[1],p2[1]
    pm=((x1+x2)/2,(y1+y2)/2)
    return pm

In [ ]:
#Determino una lista con todos los puntos medios de la oficina principal a las demas oficinas
puntos_medios=[punto_medio(coordenadas_principal[0],coordenadas[e][indices[e]]) for e in range(len(coordenadas))]

In [ ]:
#Funcion que determina la mejor ubicacion de la oficina del startup
def ubi():
    if len(puntos_medios)%2==0 & len(puntos_medios)>2:
        ubicacion=punto_medio(punto_medio(puntos_medios[0],puntos_medios[1]))
        for e in range(2,len(puntos_medios)):
            ubicacion=punto_medio(ubicacion,puntos_medios[e])
        return ubicacion
    elif len(puntos_medios)%2!=0 & len(puntos_medios)>2:
        ubicacion=punto_medio(punto_medio(puntos_medios[0],puntos_medios[1]))
        for e in range(2,len(puntos_medios)):
            ubicacion=punto_medio(ubicacion,puntos_medios[e])
        return ubicacion
    else:
        ubicacion=punto_medio(puntos_medios[0],puntos_medios[1])
        return ubicacion
    
    


In [ ]:
localization=ubi()

In [ ]:
mapa=folium.Map(location=[coordenadas_principal[0][0],coordenadas_principal[0][1]],zoom_start=3) 
folium.Circle(radius=50,location=[coordenadas_principal[0][0],coordenadas_principal[0][1]],popup=labelprincipal,color='green').add_to(mapa)
for e in range(len(entradas)):
    folium.Circle(radius=50,location=[coordenadas[e][indices[e]][0],coordenadas[e][indices[e]][1]],popup=labels[e],color='crimson').add_to(mapa)
folium.Circle(radius=50,location=[localization[0],localization[1]],popup='MEJOR UBICACION',color='blue').add_to(mapa)

In [ ]:
#https://www.geeksforgeeks.org/python-plotting-google-map-using-gmplot-package/
#https://github.com/vgm64/gmplot
#gmap.marker(hidden_gem_lat, hidden_gem_lon, 'cornflowerblue')

# Draw
#gmap.draw("my_map.html")

In [ ]:
def premisa():
    print("Hola, primero hay que definir una lista de categorias de compañias que la Startup va a necesitar posteriormente\nhay que definir la categoria de la compañia que será de mayor peso para la Startup, por ejemplo\nsi la Startup será con algo relacionado a circuitos electrónicos, Semiconductor sería la categoria principal...")
    return

In [ ]:
def Search_best_location():
    premisa()
    entradas=validation_companies()
    principal=validation_principal()
    compañias=list(map(lstcompañias,entradas))
    latitudes=list(map(lats,compañias))
    longitudes=list(map(lons,compañias))
    coordenadas=[coord(latitudes[e],longitudes[e]) for e in range(len(entradas))]
    base=entradas.index(principal)
    coordenadas_principal=coordenadas[base]
    del coordenadas[base]
    del entradas[base]
    distancias_minimas=[min(list(map(dist_principal,coordenadas[e]))) for e  in range(len(coordenadas))]
    indices=[list(map(dist_principal,coordenadas[e])).index(distancias_minimas[e]) for e in range(len(distancias_minimas))]
    labelprincipal=list(db.Companies.find({'$and':[{'category_code':principal},{'offices.latitude':coordenadas_principal[0][0]}]},{'_id':0,'name':1,'category_code':1}))
    labels=[list(db.Companies.find({'$and':[{'category_code':entradas[e]},{'offices.latitude':coordenadas[e][indices[e]][0]}]},{'_id':0,'name':1,'category_code':1})) for e in range(len(entradas))]
    puntos_medios=[punto_medio(coordenadas_principal[0],coordenadas[e][indices[e]]) for e in range(len(coordenadas))]
    localization=ubi()
    mapa=folium.Map(location=[coordenadas_principal[0][0],coordenadas_principal[0][1]],zoom_start=3) 
    folium.Circle(radius=50,location=[coordenadas_principal[0][0],coordenadas_principal[0][1]],popup=labelprincipal,color='green').add_to(mapa)
    for e in range(len(entradas)):
        folium.Circle(radius=50,location=[coordenadas[e][indices[e]][0],coordenadas[e][indices[e]][1]],popup=labels[e],color='crimson').add_to(mapa)
    folium.Circle(radius=50,location=[localization[0],localization[1]],popup='MEJOR UBICACION',color='blue').add_to(mapa)
    return mapa

In [ ]:
Search_best_location()